# Perfume Segmentation K-Means

## Setup Database

In [110]:
# Import libraries
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os

load_dotenv()

True

In [111]:
# Read database configuration from environment variables
database_config = {
  'hostname': os.getenv('DB_HOSTNAME'),
  'port': os.getenv('DB_PORT'),
  'username': os.getenv('DB_USERNAME'),
  'password': os.getenv('DB_PASSWORD'),
  'database': os.getenv('DB_DATABASE')
}

In [112]:
# Create a connection to the database
connection_str = f"mysql+pymysql://{database_config['username']}:{database_config['password']}@{database_config['hostname']}:{database_config['port']}/{database_config['database']}"
print("Connection String:", connection_str)  # Debug the connection string
engine = create_engine(connection_str)

Connection String: mysql+pymysql://root:@10.2.18.133:3306/perfume_segmentation


In [113]:
# Load perfumes table
query = """
SELECT
    s.id,
    s.age,
    s.gender,
    s.profession_id,
    s.perfume_id,
    p.name AS perfume_name,
    pr.name AS profession_name
FROM sales AS s
LEFT JOIN perfumes AS p ON s.perfume_id = p.id
LEFT JOIN professions AS pr ON s.profession_id = pr.id
"""
df_sales = pd.read_sql(query, engine)

df_sales.head()


,id,age,gender,profession_id,perfume_id,perfume_name,profession_name
0,1,54,0,0,0,Bacarat,Ibu Rumah Tangga
1,2,49,0,0,1,D&G,Ibu Rumah Tangga
2,3,26,0,1,2,Paris Hilton,Pegawai Negeri
3,4,40,0,2,1,D&G,Karyawan
4,5,52,0,2,0,Bacarat,Karyawan


## Preprocessing

In [120]:
from sklearn.cluster import KMeans

In [ ]:
# Select features
X = df_sales[['age', 'gender', 'profession_id', 'perfume_id']].copy()

# Init KMeans
k = 5  # Clusters total
kmeans = KMeans(n_clusters=k, random_state=42)

# Train KMeans
kmeans.fit(X)

# KMeans labels
labels = kmeans.labels_
df_sales['cluster'] = labels

df_sales['cluster'].head()


0    1
1    1
2    0
3    2
4    1
Name: cluster, dtype: int32

In [123]:
# Get cluster centroids
centroids = kmeans.cluster_centers_

# Get cluster labels
df_sales.groupby('cluster').agg({
    'age': 'mean',
    'gender': 'mean',
    'profession_id': 'mean',
    'perfume_id': 'mean',
    'id': 'count'
})

,age,gender,profession_id,perfume_id,id
cluster,,,,,
0,23.573529,0.529412,3.279412,1.470588,68
1,51.912281,0.368421,1.842105,1.403509,57
2,35.790698,0.395349,1.883721,1.697674,43
3,16.122449,0.489796,2.897959,1.897959,49
